In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import datetime
import requests
import os
from cred import BRICKS_CONFIG, DB_CONFIG
from params import firefly_table_list


DB_URL = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(DB_URL)

def get_all_tables():
    """List all tables in the database."""
    inspector = inspect(engine)
    return inspector.get_table_names()

def extract_and_upload():
    tables = firefly_table_list
    print(f"{len(tables)} tables were found to be processed.")

    for table_name in tables:
        try:
            print(f"--- Processing: {table_name} ---")
            
            # 1. Extração
            df = pd.read_sql(f"SELECT * FROM {table_name}", engine)
            df['extracted_at'] = datetime.datetime.now()
            
            file_name = f"{table_name}.parquet"
            df.to_parquet(
                file_name, 
                index=False, 
                use_deprecated_int96_timestamps=True
            )

            # 2. Upload to Volume
            url = f"{BRICKS_CONFIG['workspace_url']}/api/2.0/fs/files/Volumes/{BRICKS_CONFIG['catalog']}/{BRICKS_CONFIG['schema']}/{BRICKS_CONFIG['volume']}/{file_name}"
            headers = {"Authorization": f"Bearer {BRICKS_CONFIG['token']}"}
            
            with open(file_name, 'rb') as f:
                response = requests.put(url, headers=headers, data=f, params={"overwrite": "true"})
            
            if response.status_code in [200, 204]:
                print(f"SUCCESS: {table_name} sent.")
            else:
                print(f"ERROR {table_name}: {response.text}")
            
            # cleanup local file
            os.remove(file_name)

        except Exception as e:
            print(f"FAIL {table_name}: {e}")

if __name__ == "__main__":
    extract_and_upload()

9 tables were found to be processed.
--- Processing: transactions ---
SUCCESS: transactions sent.
--- Processing: transaction_journals ---
SUCCESS: transaction_journals sent.
--- Processing: accounts ---
SUCCESS: accounts sent.
--- Processing: transaction_types ---
SUCCESS: transaction_types sent.
--- Processing: category_transaction_journal ---
SUCCESS: category_transaction_journal sent.
--- Processing: categories ---
SUCCESS: categories sent.
--- Processing: budget_transaction_journal ---
SUCCESS: budget_transaction_journal sent.
--- Processing: budgets ---
SUCCESS: budgets sent.
--- Processing: bills ---
SUCCESS: bills sent.
